# Task1: Get detections from Mask R-CNN

This notebook is based on this [official demo](https://github.com/facebookresearch/maskrcnn-benchmark/blob/master/demo/Mask_R-CNN_demo.ipynb) and it can be run in a docker container powered with Jupyter. 

A Dockerfile can be found in the [official repository](https://github.com/facebookresearch/maskrcnn-benchmark/blob/master/docker/docker-jupyter/Dockerfile) but it needs to be modified to fix an issue.

###  Instructions

1. Add the following line at the end of the Dockerfile:
    
    `RUN pip install requests`
    
1. Build image with:

     `docker build -t my-maskrcnn-benchmark-jupyter --build-arg CUDA=10.0 --build-arg CUDNN=7 docker/docker-jupyter/`

1. Copy your dataset into _demo_ directory

1. Run a docker container using the image:
    
    `docker run -dt -p 8888:8888 --name jupyter -v $PWD:/notebooks/my my-maskrcnn-benchmark-jupyter`

1. Retrieve Jupyter notebook token using:
    
    `docker logs jupyter`
    

## Setup inference

In [4]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

import requests
from io import BytesIO
from PIL import Image
import numpy as np

In [5]:
# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 20, 12

In [6]:
from maskrcnn_benchmark.config import cfg
from predictor import COCODemo

In [7]:
config_file = "../configs/caffe2/e2e_mask_rcnn_R_50_FPN_1x_caffe2.yaml"
# update the config options with the config file
cfg.merge_from_file(config_file)
# manual override some options
cfg.merge_from_list(["MODEL.DEVICE", "cpu"])

Now we create the `COCODemo` object. It contains a few extra options for conveniency, such as the confidence threshold for detections to be shown.

In [8]:
coco_demo = COCODemo(
    cfg,
    min_image_size=800,
    confidence_threshold=0.7,
)

Let's define a few helper functions for loading images from a URL

In [9]:
def load(url):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    response = requests.get(url)
    pil_image = Image.open(BytesIO(response.content)).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")

## Run prediction over all images

Read images

In [ ]:
from pathlib import Path

def image_iterator(paths) -> np.ndarray:
    for path in paths:
        image = Image.open(path)
        image = np.array(image)[:, :, [2, 1, 0]]
        yield image

AICITY_DIR = Path('/notebooks/my/demo/data/AICity_data/train/S03/c010')
frame_paths = AICITY_DIR.glob('frames/*')
images = image_iterator(frame_paths)

Define helpers

In [ ]:
def formatme(detection):
    return (f'{int(detection[0])},'
            f'{int(detection[1])},'
            f'{detection[2]:.3f},'
            f'{detection[3]:.3f},'
            f'{detection[4]:.3f},'
            f'{detection[5]:.3f},'
            f'{detection[6]:.3f},'
            f'{int(detection[7])},'
            f'{int(detection[8])},'
            f'{int(detection[9])}'
    )

In [41]:
import torch

output_path = Path('/notebooks/my/demo/det_mask_rcnn.txt')

with output_path.open('w') as file:
    for frame_num, image in enumerate(images, 1):
        predictions = coco_demo.compute_prediction(image)
        # Select only 'car' (category=3) predictions
        cars = predictions[predictions.get_field('labels') == 3]

        # Build a tensor with [labels, bbox, scores]
        scores = cars.get_field('scores')
        labels = cars.get_field('labels').float()
        preds = torch.stack([
                labels, 
                cars.bbox[:,0],
                cars.bbox[:,1],
                cars.bbox[:,2],
                cars.bbox[:,3],
                scores, 
            ], dim=1)

        # Format data
        ones = torch.ones((preds.shape[0], 1))
        none = - ones
        frame_num = frame_num * ones
        data = torch.cat([frame_num, preds, - ones, -ones, -ones], dim=1)
        lines = data.tolist()
        
        # Write to file
        for detection in lines:
            string = formatme(detection)
            file.write(string + '\n')